# HuggingFace Tutorial

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import torch

from transformers import AlbertTokenizer, AlbertModel

## Tokenizer

In [ ]:

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertModel.from_pretrained("albert-base-v2")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [ ]:
encoded_input

In [ ]:
print(output.last_hidden_state.shape)
tokenizer.decode(encoded_input["input_ids"][0])

## Datasets

In [17]:
from datasets import load_dataset, DatasetDict

In [11]:
dataset = load_dataset("yelp_review_full")

In [12]:
dataset["train"][100]
# dataset_sm = DatasetDict({
#   "train": dataset["train"].select(range(1000)),
#   "test": dataset["test"].select(range(1000)),
# })

# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [25]:
def tokenize_function(examples: dict[str, list[int | str]]):
  """Examples is a `dict`-like object with a `label` and `text` field.
  
  Each field has a list of integers (for `label`) or strings (for `text`).
  """
  return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset_sm.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Training

In [5]:
import torch

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments
from transformers import TrainingArguments, Trainer

from datasets import load_dataset, DatasetDict

import numpy as np
import evaluate

In [6]:
model_name = 'albert-base-v2'

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
tokenizer = AutoTokenizer.from_pretrained(model_name)

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred: tuple[torch.Tensor, torch.Tensor]):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)


def tokenize_function(examples: dict[str, list[int | str]]):
  """Examples is a `dict`-like object with a `label` and `text` field.
  
  Each field has a list of integers (for `label`) or strings (for `text`).
  """
  return tokenizer(examples["text"], padding="max_length", truncation=True)


dataset = load_dataset("yelp_review_full")
dataset_mini_size = 1000
dataset_mini = DatasetDict({
  "train": dataset["train"].select(range(dataset_mini_size)),
  "test": dataset["test"].select(range(dataset_mini_size)),
})

tokenized_dataset_mini = dataset_mini.map(tokenize_function, batched=True)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_dataset_mini["train"],
  eval_dataset=tokenized_dataset_mini["test"],
  compute_metrics=compute_metrics,
)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
trainer.train()

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.6027281284332275, 'eval_accuracy': 0.228, 'eval_runtime': 41.3642, 'eval_samples_per_second': 24.176, 'eval_steps_per_second': 3.022, 'epoch': 1.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.609639286994934, 'eval_accuracy': 0.204, 'eval_runtime': 37.6814, 'eval_samples_per_second': 26.538, 'eval_steps_per_second': 3.317, 'epoch': 2.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.613320231437683, 'eval_accuracy': 0.204, 'eval_runtime': 37.9326, 'eval_samples_per_second': 26.363, 'eval_steps_per_second': 3.295, 'epoch': 3.0}
{'train_runtime': 451.3187, 'train_samples_per_second': 6.647, 'train_steps_per_second': 0.831, 'train_loss': 1.6177283528645834, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=1.6177283528645834, metrics={'train_runtime': 451.3187, 'train_samples_per_second': 6.647, 'train_steps_per_second': 0.831, 'train_loss': 1.6177283528645834, 'epoch': 3.0})

## Evaluation

In [9]:
import evaluate
accuracy =  evaluate.load("accuracy")
accuracy.compute(predictions=[0, 1, 1, 0], references=[0, 1, 0, 1])

{'accuracy': 0.5}